# Tutorial: How to use PyLKMotor

## Introduction


PyLKMotor is a Python library that provides a simple interface to control the LK motors. This tutorial will show you how to use PyLKMotor to control the LK motors.

## Installation

You can install PyLKMotor using pip:

```bash
pip install pylkmotor
```

Or you can install it from the source code:

```bash
git clone https://github.com/han-xudong/pyLKMotor.git
cd pylkmotor
pip install .
```

## Initialization

To use PyLKMotor, you need to import the `LKMotor` class from the `pylkmotor` module, and create an instance of the `LKMotor` class. You need to specify the can bus interface, bus channel, and the motor id when creating the `LKMotor` instance.

In [ ]:
from pylkmotor import LKMotor

motor = LKMotor(bus_interface="socketcan", bus_channel="can0", motor_id=1)

## Read motor status

PyLKMotor provides several functions to read the motor status, including:

### `read_motor_status_1`

This function sends a command to the motor to read the motor status including:
- Motor temperature (int8_t, 1 °C/LSB)
- Motor voltage (int16_t, 0.01 V/LSB)
- Motor current (int16_t, 0.01 A/LSB)
- Motor state (uint8_t)
- Error state (uint8_t)

In [ ]:
temperature, voltage, current, motor_state, error_state = motor.read_motor_status_1()
print(f"Temperature: {temperature} °C")
print(f"Voltage: {voltage} V")
print(f"Current: {current} A")
print(f"Motor state: {motor_state}")
print(f"Error state: {error_state}")

### `read_motor_status_2`

This function sends a command to the motor to read the motor status including:
- Motor temperature (int8_t, 1 °C/LSB)
- Motor iq (int16_t)
- Motor speed (int16_t, 1 dps/LSB)
- Encoder value (uint16_t)

In [ ]:
temperature, iq, speed, encoder_val = motor.read_motor_status_2()
print(f"Temperature: {temperature} °C")
print(f"Iq: {iq}")
print(f"Speed: {speed} deg/s")
print(f"Encoder value: {encoder_val}")

### `read_motor_status_3`

This function sends a command to the motor to read the motor status including:
- Motor temperature (int8_t, 1 °C/LSB)
- Phase A, B, C current (int16_t)

In [ ]:
temperature, current_A, current_B, current_C = motor.read_motor_status_3()
print(f"Motor temperature: {temperature} °C")
print(f"Current A: {current_A} A")
print(f"Current B: {current_B} A")
print(f"Current C: {current_C} A")

## Read and set encoder value

The LK motor has an encoder to measure the motor position. You can read and set the encoder value using the following functions:

### `read_encoder_data`

This function sends a command to the motor to read the encoder data including:
- Encoder value (uint16_t)
- Raw encoder value (uint16_t)
- Encoder offset (uint16_t)

In [ ]:
encoder_val, encoder_raw, encoder_offset = motor.read_encoder()
print(f"Encoder value: {encoder_val}")
print(f"Encoder raw value: {encoder_raw}")
print(f"Encoder offset: {encoder_offset}")

### `set_zero_position`

This function sends a command to the motor to set the current position as the zero position.

**Note**: the encoder offset will be updated until the motor is restarted.

**Warning**: this command will effect the lifetime of the driver, do not use it frequently.

In [ ]:
motor.set_zero_position()

### `read_multi_turn_angle`

This function sends a command to the motor to read the multi-turn angle (int64_t, 0.01 degree/LSB).

In [ ]:
multi_turn_angle = motor.read_multi_turn_angle()
print(f"Multi-turn angle: {multi_turn_angle/100} °")

### `read_single_turn_angle`

This function sends a command to the motor to read the single-turn angle (int16_t, 0.01 degree/LSB, 0-35999).

In [ ]:
single_turn_angle = motor.read_single_turn_angle()
print(f"Single-turn angle: {single_turn_angle/100} °")

### `set_position_to_angle`

This function sends a command to the motor to set the current position as a multi-turn angle (int64_t, 0.01 degree/LSB). It's recommended to use this function to set the position instead of `set_zero_position`.

In [ ]:
motor.set_position_to_angle(0)

## Control the motor

### Run, stop, and shutdown

There are three functions to change the motor state:

- `run_motor`: Run the motor. The LED light will keep on. The motor can receive commands and execute them.
- `stop_motor`: Stop the motor. The state of the motor will not be cleared. The motor can receive new commands and execute them.
- `shutdown_motor`: Turn off the motor. It will clear the number of turns and previous commands. The LED light will shine slowly. The motor can receive and respond to the commands, but does not execute them.


When then motor is powered on, the default state is `RUN`. When you send `stop_motor` command, the motor will stop, but the state is still `RUN`. If you send `shutdown_motor` command, the motor will turn off, and the state will be `SHUTDOWN` until you send `run_motor` command.

In [ ]:
motor.motor_run()

In [ ]:
motor.motor_stop()

In [ ]:
motor.motor_shutdown()

### Torque control

The `torque_loop_control` function sends a command to the motor to control the motor torque. The motor will keep the torque until you send a new command. The input `iq_control` is the torque control value (int16_t, -2048 to 2048).

In [ ]:
motor.torque_loop_control(iq_control=100)

### Speed control

The `speed_loop_control` function sends a command to the motor to control the motor speed. The motor will keep the speed until you send a new command. The input `iq_control` is the torque control value (int16_t, -2048 to 2048), and `speed_control` is the speed control value (int32_t, 0.01 dps/LSB).

In [ ]:
motor.speed_loop_control(iq_control=100, speed_control=1000)

### Position control

#### Multi-turn position control

This function sends a command to the motor to control the motor in multi-turn position loop. The input `angle_control` is the target angle (int32_t, 0.01 degree/LSB), and `max_speed` (optional) is the maximum speed (int16_t, 1 dps/LSB).

In [ ]:
motor.multi_turn_position_control(angle_control=10000, max_speed=1000)

#### Single-turn position control

This function sends a command to the motor to control the motor in single-turn position loop. The input `spin_direction` is the spin direction (uint8_t, 0x00: clockwise, 0x01: counterclockwise), `angle_control` is the target angle (int32_t, 0.01 degree/LSB, 0-35999), and `max_speed` (optional) is the maximum speed (int16_t, 1 dps/LSB).

In [ ]:
motor.single_turn_position_control(angle_control=10000, max_speed=1000)

#### Incremental position control

This function sends a command to the motor to control the motor in incremental position loop. The input `angle_increment` is the target angle increment (int32_t, 0.01 degree/LSB), and `max_speed` (optional) is the maximum speed (int16_t, 1 dps/LSB).

In [ ]:
motor.incremental_position_control(angle_increment=1000, max_speed=1000)

## Read and set parameters

PyLKMotor provides functions to read and set the control parameters:

- `read_control_params`: Read the motor parameter.
- `write_control_params`: Write the motor parameter.

**Note**: The control parameters are stored in the motor, please be careful to follow the instructions in the [LK Motor User Manual](can_protocol.pdf) to change the parameters.